In [0]:
greed_god_data = [
    (1,'Zeus','M'),
    (2,'Ares','M'),
    (3,'Hera','F'),
    (4,'Persephone','F'),
    (5,'Demeter','F')
] 

greed_god_schema = ['g_id','g_name','g_gender']

greek_god_df = spark.createDataFrame(greed_god_data,greed_god_schema)

relation_data = [
(2,1),
(2,3),
(4,1),
(4,5)
]

relation_schema = ['c_id','p_id']

relation_df = spark.createDataFrame(relation_data,relation_schema)

In [0]:
from pyspark.sql.functions import col,when

In [0]:
child_to_father_df = relation_df.join(greek_god_df,relation_df.p_id == greek_god_df.g_id,'inner')\
                    .withColumn('parent_gender',when(col('g_gender') == 'M','Male').otherwise('Female'))\
                    .withColumn('Father',when(col('parent_gender')=='Male',col('g_name')).otherwise(None))\
                    .select('c_id','Father').filter(col('Father').isNotNull())

In [0]:
child_to_mother_df = relation_df.join(greek_god_df,relation_df.p_id == greek_god_df.g_id,'inner')\
                    .withColumn('parent_gender',when(col('g_gender') == 'F','Female').otherwise('Male'))\
                    .withColumn('Mother',when(col('parent_gender')=='Female',col('g_name')).otherwise(None))\
                    .select('c_id','Mother').filter(col('Mother').isNotNull())

In [0]:
final_df = greek_god_df.join(child_to_father_df,greek_god_df.g_id == child_to_father_df.c_id,'inner').join(child_to_mother_df,greek_god_df.g_id == child_to_mother_df.c_id,'inner').select(col('g_name').alias('child'),'Father','Mother').show()

In [0]:
data = [
    (1,2023,10000),
    (1,2024,12000),
    (1,2025,12000),
    (2,2023,10000),
    (2,2024,10000)
]

schema = ['id','year','salary']

test_df = spark.createDataFrame(data,schema)

In [0]:
from pyspark.sql.functions import col, lead
from pyspark.sql.window import Window

window = Window.partitionBy('id').orderBy(col('year').desc())

new_test_df = test_df.withColumn('rank',lead(col('salary'),1).over(window))

result_df = new_test_df.withColumn('difference',col('salary')-col('rank'))

final_df = result_df.filter(col('difference') > 0).select(col('id'),col('difference'))

display(final_df)

In [0]:
son_father_data = [
    ('Ram','Dasrath'),
    ('Laxman','Dasrath'),
    ('Bharat','Dasrath'),
    ('Shatrughna','Dasrath'),
    ('Bharat','Vishwamirtra')
]

son_father_schema = ['son','father']

son_father_df = spark.createDataFrame(son_father_data,son_father_schema)

In [0]:
from pyspark.sql.functions import count

son_count_df = son_father_df.groupBy('son').agg(count('father').alias('count'))

display(son_count_df)

In [0]:
numbers = [1, 2, 3, 5, 6]

missing_number = set(range(numbers[0], numbers[-1] + 1)) - set(numbers)
missing_number = list(missing_number)[0]

missing_number

In [0]:
comments_data = [
    ('excellent',),
    ('food is excellent',),
    ('service was ok',),
    ('ambience was excellent, food was excellent, service was ok',)
]
comments_schema = ['comment']

comments_df = spark.createDataFrame(comments_data,comments_schema)

In [0]:
from pyspark.sql.functions import col, lower,split, explode, count

excellent_count = comments_df.withColumn('comment',explode(split(col('comment'),','))).withColumn('comment',explode(split(col('comment'),' ')))

df_exc = excellent_count.filter(lower(col('comment')) == 'excellent')

df_exc_count = df_exc.groupBy().agg(count('comment').alias('count'))

display(df_exc_count)

In [0]:
fruit_data = [
    ('apple',10),
    ('banana',5),
    ('apple',20),
    ('orange',15)
]

fruit_schema = ['fruit','quantity']

fruit_df = spark.createDataFrame(fruit_data,fruit_schema)

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

window = Window.partitionBy('fruit').orderBy(col('quantity'))

unique_fruit = fruit_df.withColumn('unique_quantity',row_number().over(window))

display(unique_fruit)